<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/law-staging/Notebooks/How_to_use_nested_tables_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Nested Tables in BigQuery

## Introduction

The data that GDC provides through their API is naturally nested. A given patient may, for instance, have multiple diagnoses at the same time. More frequently, they may receive multiple treatments to combat their cancer. Usually, they would have multiple follow up visits to monitor their cancer's progression. The data generated in these examples means that there will be a variable number of records of each type for a given research subject. Traditionally, we have stored these supplemental records in a separate table. However, they are extracted from GDC in the form of a nested JSON object.

BigQuery supports nested columns, and there are advantages to storing the data in this format. Since all of the data is stored in one table, you don't have to locate and join the supplemental tables in order to access data. Instead, you can use UNNEST clauses to flatten the data as needed for your research. This guide provides an introduction to using UNNEST in your SQL queries.

## Getting Started

Here, we import library modules necessary to run our script, including the bigquery module.

In [ ]:
import time
import datetime
import typing

from google.cloud import bigquery

# When receiving a BQ result, we can convert each record row to a dict.
# This is the typing definition for the resulting object.
BigQueryRowObject = dict[str, typing.Union[str, bool, int, float, datetime.datetime, None, dict]]

Next we will need to Authorize our access to BigQuery and the Google Cloud. For more information, see '[Quick Start Guide to ISB-CGC](https://https://nbviewer.org/github/isb-cgc/Community-Notebooks/blob/master/Notebooks/Quick_Start_Guide_to_ISB_CGC.ipynb)'. Alternative authentication methods can be found [here](https://googleapis.dev/python/google-api-core/latest/auth.html).

In [ ]:
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=6uTbS3YyAQzzmsfhPzK7nb0tV8v9qc&prompt=consent&access_type=offline&code_challenge=E0FGXjz13qLOUfwKdBnttL_TbgdQwtvQnNo-3nFqOcc&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhUvoKkY0wQuaZh69wcPm9ZekoGuUmr0UErT_WHPklCjdMC8ia2jj5aY89R_QP4g6A

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to AD

## Example 1: Retrieve treatment_ids for a Given Patient

Say that we want to retrieve all of the treatment_ids associated with a given patient. There are multiple ways we could achieve that. One option is to use a BigQuery QueryJob to retrieve the patient record, then use python to locate all the treatment_ids, as shown below:

In [ ]:
def get_query_result(sql: str) -> typing.Union[bigquery.table.RowIterator, None]:
  # Executes a given SQL statement and returns the query result.

  client = bigquery.Client('isb-cgc-CHANGE-ME')
  location = 'US'

  # initialize the QueryJob
  job_config = bigquery.QueryJobConfig()

  # execute the query
  query_job = client.query(query=sql, location=location, job_config=job_config)

  while query_job.state != 'DONE':
    query_job = client.get_job(query_job.job_id, location=location)
    # wait for the job to complete
    if query_job.state != 'DONE':
      time.sleep(5)

  query_job = client.get_job(query_job.job_id, location=location)

  if query_job.error_result is not None:
    print(f'[ERROR] {query_job.error_result}')
    return None

  # Return the query result as a RowIterator object
  return query_job.result()


sql = """
  SELECT *
  FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37`
  WHERE case_id = 'c3f876f4-2d3a-4d60-b6c4-019f94010330'
"""

query_result = get_query_result(sql=sql)

case_record_list = list()

for case_row in query_result:
  # convert RowIterator object into a list of dictionaries
  case_record = dict(case_row.items())
  case_record_list.append(case_record)

for case_record in case_record_list:
  diagnosis_list = case_record['diagnoses']

  for diagnosis in diagnosis_list:
    treatment_list = diagnosis['treatments']

    for treatment in treatment_list:
      print(treatment['treatment_id'])

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Job 3407dfac-0b1f-4e61-a9f5-ee0f8cff342b is done

0630e200-4d21-4700-b449-ce4401fb74e4
10b7f95e-9536-476c-a50e-58d16309e92e
3ae75304-26ba-5147-a742-ba35a1925afa
58e72cd9-0de5-57fd-a7d7-7de1481366eb
bcc4de68-8732-41c4-b8db-1873bed0c1d3
e0cd977a-2919-4ff8-9d65-bdf49526b7a6
e4978478-c4aa-41e2-9848-7691cf4b9711


An alternative, and simpler, approach is to use an UNNEST clause to retrieve the treatment_ids. This example employs the bigquery magic command, which you can learn more about [here](https://notebook.community/GoogleCloudPlatform/python-docs-samples/notebooks/tutorials/bigquery/BigQuery%20query%20magic).

In [ ]:
%%bigquery --project isb-cgc-CHANGE-ME
SELECT treatment.treatment_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37`,
UNNEST(diagnoses) AS diagnosis, # first we unnest diagnoses to access its columns
UNNEST(treatments) AS treatment # then we unnest treatments, a child column of diagnoses
WHERE case_id = 'c3f876f4-2d3a-4d60-b6c4-019f94010330'


Query is running:   0%|          |

Downloading:   0%|          |

,treatment_id
0,0630e200-4d21-4700-b449-ce4401fb74e4
1,10b7f95e-9536-476c-a50e-58d16309e92e
2,3ae75304-26ba-5147-a742-ba35a1925afa
3,58e72cd9-0de5-57fd-a7d7-7de1481366eb
4,bcc4de68-8732-41c4-b8db-1873bed0c1d3
5,e0cd977a-2919-4ff8-9d65-bdf49526b7a6
6,e4978478-c4aa-41e2-9848-7691cf4b9711


## Example 2: Retrieve submitter_ids from Diagnoses and Treatments

In the query below, we select a patient record using the case_id, then use UNNEST to retrieve the diagnosis and treatment submitter_ids.

Note: diagnoses and treatments both contain a column named submitter_id, which creates a naming conflict when unnested. You can address this by explicitly renaming the columns, as shown in the example below. If you don't, BigQuery will append an integer suffix to every duplicate column name. (e.g. submitter_id, submitter_id_1, etc.)

In [ ]:
%%bigquery --project isb-cgc-CHANGE-ME
SELECT diagnosis.submitter_id AS diagnosis_submitter_id,
  treatment.submitter_id AS treatment_submitter_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37` AS base_case,
UNNEST(diagnoses) AS diagnosis
LEFT JOIN UNNEST(diagnosis.treatments) AS treatment
WHERE case_id = '39dce88d-112c-4a3d-b2d2-11e0616594d8'

Query is running:   0%|          |

Downloading:   0%|          |

,diagnosis_submitter_id,treatment_submitter_id
0,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_15
1,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_11
2,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_13
3,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_14
4,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_18
5,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_1
6,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_6
7,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_7
8,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_17
9,BLGSP-71-06-00252_diagnosis,BLGSP-71-06-00252_treatment_2


## Example 3: Unnesting Multiple Column Groups

The case we used in the above query doesn't have any follow_up records, so let's look at a different `case_id`: `18395371-3c84-4d39-8ace-a3546e9ea34e`

If we unnest all three nested columns, the resulting output will actually be the Cartesian product, representing every possible combination of the diagnosis, treatment and follow_up records.

The new case has one diagnosis record, 8 treatment records, and 6 follow_up records. $1*8*6=48$. Our result will have 48 rows.

The best way to avoid this is to unnest one set of nested columns at a time, e.g. diagnoses (and optionally its children: diagnoses.treatments, diagnoses.pathlogy_details, diagnoses.annotations).

In [ ]:
%%bigquery --project isb-cgc-CHANGE-ME
SELECT base_case.case_id,
  diagnosis.diagnosis_id,
  treatment.treatment_id,
  follow_up.follow_up_id
FROM `isb-cgc-bq.CGCI_versioned.clinical_nested_gdc_r37` AS base_case
LEFT JOIN UNNEST(diagnoses) AS diagnosis
LEFT JOIN UNNEST(diagnosis.treatments) AS treatment
LEFT JOIN UNNEST(follow_ups) AS follow_up
WHERE case_id = '18395371-3c84-4d39-8ace-a3546e9ea34e'

Query is running:   0%|          |

Downloading:   0%|          |

,case_id,diagnosis_id,treatment_id,follow_up_id
0,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,3eb27fdc-a254-4edd-b271-4062e09b1649
1,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,465ed8a1-d0b2-4960-b7e9-611dea4fd952
2,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,55d5a520-101b-44e8-80a2-b622b59f42ec
3,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,a3d33feb-f2db-4745-8e28-1c2c78093f5f
4,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,c41c0549-2d14-4710-91dc-ed0aef3d2e8f
5,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,77f3a3b5-8bf7-4a53-95ed-15079795103e,d59e772a-43ea-4ab9-bd02-eef48372ad77
6,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,3eb27fdc-a254-4edd-b271-4062e09b1649
7,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,465ed8a1-d0b2-4960-b7e9-611dea4fd952
8,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,55d5a520-101b-44e8-80a2-b622b59f42ec
9,18395371-3c84-4d39-8ace-a3546e9ea34e,20cbc3e8-86f1-43c9-9b6a-d5a85f027685,98e3a1df-6ca3-4e2f-a453-ca2da4c3401b,a3d33feb-f2db-4745-8e28-1c2c78093f5f
